# Database Check
Quick validation of database health after repopulation.

| Cell | Description |
|------|-------------|
| 1 | Setup - imports and database connection |
| 2 | Quick table overview with row counts |
| 3 | Data Completeness - newest/oldest 5 days + NULL flagging |
| 4 | Multi-Table Health Summary - freshness check |
| 5 | Consolidated Sanity Checks - duplicates, invalid values |
| 6 | Cleanup - close connection |

**Usage:** Change `check_symbol` in Cell 3 to test different tickers.

In [ ]:
# Cell 1: Setup - imports and database connection
import sys
from pathlib import Path

# Add darkpool root to path for darkpool_analysis imports
darkpool_root = Path(__file__).parent.parent if '__file__' in dir() else Path.cwd().parent
sys.path.insert(0, str(darkpool_root))

from darkpool_analysis.config import load_config
from darkpool_analysis.db import get_connection
import pandas as pd

config = load_config()
conn = get_connection(config.db_path)

In [ ]:
# Cell 2: Quick Table Overview - all tables with row counts
from IPython.display import display

overview = conn.execute("""
    SELECT name as table_name
    FROM (SHOW TABLES)
""").df()

# Add row counts
row_counts = []
for tbl in overview['table_name']:
    cnt = conn.execute(f"SELECT COUNT(*) as n FROM {tbl}").df()['n'].iloc[0]
    row_counts.append(cnt)
overview['rows'] = row_counts
display(overview)

In [ ]:
# Cell 3: Data Completeness Check - newest/oldest 5 days with NULL field flagging

check_symbol = "AAPL"  # Change to test different tickers

print(f"{'='*60}")
print(f"DATA COMPLETENESS CHECK FOR: {check_symbol}")
print(f"{'='*60}")

# Get newest 5 days
print("\n📅 NEWEST 5 DAYS (daily_metrics):")
newest = conn.execute("""
    SELECT * FROM daily_metrics
    WHERE symbol = ?
    ORDER BY date DESC
    LIMIT 5
""", [check_symbol]).df()
display(newest)

# Get oldest 5 days
print("\n📅 OLDEST 5 DAYS (daily_metrics):")
oldest = conn.execute("""
    SELECT * FROM daily_metrics
    WHERE symbol = ?
    ORDER BY date ASC
    LIMIT 5
""", [check_symbol]).df()
display(oldest)

# Combine for NULL analysis
combined = pd.concat([newest, oldest], ignore_index=True)

# Count NULLs per column
print("\n🔍 NULL FIELD ANALYSIS (across newest + oldest 10 rows):")
null_counts = combined.isnull().sum()
total_rows = len(combined)

# Create summary dataframe
null_summary = pd.DataFrame({
    'column': null_counts.index,
    'null_count': null_counts.values,
    'total_rows': total_rows,
    'null_pct': (null_counts.values / total_rows * 100).round(1),
    'status': ['⚠️ MISSING' if n > 0 else '✅ OK' for n in null_counts.values]
})

# Show only columns with issues first, then OK columns
null_summary_sorted = null_summary.sort_values(['null_count', 'column'], ascending=[False, True])
display(null_summary_sorted)

# Summary flag
missing_cols = null_summary[null_summary['null_count'] > 0]['column'].tolist()
if missing_cols:
    print(f"\n⚠️  ALERT: {len(missing_cols)} columns have NULL values:")
    for col in missing_cols:
        pct = null_summary[null_summary['column'] == col]['null_pct'].values[0]
        print(f"   - {col}: {pct}% null")
else:
    print("\n✅ ALL FIELDS POPULATED - No NULL values detected")

In [ ]:
# Cell 4: Multi-Table Data Health Summary - check all key tables for sample ticker
print(f"{'='*60}")
print(f"MULTI-TABLE HEALTH CHECK FOR: {check_symbol}")
print(f"{'='*60}")

# Define tables and their date columns
table_checks = [
    ("daily_metrics", "date", "symbol"),
    ("lit_direction_daily", "date", "symbol"),
    ("finra_short_daily_raw", "trade_date", "symbol"),
    ("polygon_daily_agg_raw", "trade_date", "symbol"),
]

health_rows = []

for table, date_col, sym_col in table_checks:
    try:
        result = conn.execute(f"""
            SELECT 
                COUNT(*) as row_count,
                MIN({date_col}) as oldest_date,
                MAX({date_col}) as newest_date
            FROM {table}
            WHERE {sym_col} = ?
        """, [check_symbol]).df()
        
        row_count = result['row_count'].iloc[0]
        oldest = result['oldest_date'].iloc[0]
        newest = result['newest_date'].iloc[0]
        
        # Check if newest date is within last 7 days
        if newest and pd.Timestamp(newest) >= pd.Timestamp.now() - pd.Timedelta(days=7):
            freshness = "✅ FRESH"
        elif newest:
            freshness = "⚠️ STALE"
        else:
            freshness = "❌ NO DATA"
            
        health_rows.append({
            'table': table,
            'rows': row_count,
            'oldest': oldest,
            'newest': newest,
            'freshness': freshness
        })
    except Exception as e:
        health_rows.append({
            'table': table,
            'rows': 0,
            'oldest': None,
            'newest': None,
            'freshness': f"❌ ERROR: {str(e)[:30]}"
        })

health_df = pd.DataFrame(health_rows)
display(health_df)

# Overall verdict
stale_tables = health_df[health_df['freshness'].str.contains('STALE|ERROR|NO DATA', na=False)]['table'].tolist()
if stale_tables:
    print(f"\n⚠️  ATTENTION: {len(stale_tables)} table(s) may need refresh:")
    for t in stale_tables:
        print(f"   - {t}")
else:
    print("\n✅ ALL TABLES HAVE FRESH DATA")

In [ ]:
# Cell 5: Consolidated Sanity Checks - duplicates and invalid values
print(f"{'='*60}")
print("SANITY CHECKS")
print(f"{'='*60}")

# Check 1: Duplicate rows in daily_metrics
print("\n🔍 Duplicate rows in daily_metrics:")
dupes_dm = conn.execute("""
    SELECT symbol, date, COUNT(*) AS n
    FROM daily_metrics
    GROUP BY symbol, date
    HAVING n > 1
""").df()
if len(dupes_dm) == 0:
    print("   ✅ No duplicates found")
else:
    print(f"   ⚠️ {len(dupes_dm)} duplicate(s) found:")
    display(dupes_dm)

# Check 2: Duplicate rows in lit_direction_daily
print("\n🔍 Duplicate rows in lit_direction_daily:")
dupes_lit = conn.execute("""
    SELECT symbol, date, COUNT(*) AS n
    FROM lit_direction_daily
    GROUP BY symbol, date
    HAVING n > 1
""").df()
if len(dupes_lit) == 0:
    print("   ✅ No duplicates found")
else:
    print(f"   ⚠️ {len(dupes_lit)} duplicate(s) found:")
    display(dupes_lit)

# Check 3: Invalid short ratios (outside 0-1 range)
print("\n🔍 Invalid short ratios (outside 0-1):")
bad_ratios = conn.execute("""
    SELECT symbol, date, short_ratio
    FROM daily_metrics
    WHERE short_ratio IS NOT NULL AND (short_ratio < 0 OR short_ratio > 1)
    LIMIT 10
""").df()
if len(bad_ratios) == 0:
    print("   ✅ All short ratios valid")
else:
    print(f"   ⚠️ Invalid ratios found:")
    display(bad_ratios)

# Check 4: Zero/negative lit volumes with non-null ratios
print("\n🔍 Invalid lit volumes (zero/negative with computed ratio):")
bad_lit = conn.execute("""
    SELECT symbol, date, lit_buy_volume, lit_sell_volume
    FROM lit_direction_daily
    WHERE (lit_buy_volume <= 0 OR lit_sell_volume <= 0)
      AND log_buy_sell IS NOT NULL
    LIMIT 10
""").df()
if len(bad_lit) == 0:
    print("   ✅ All lit volumes valid")
else:
    print(f"   ⚠️ Invalid volumes found:")
    display(bad_lit)

print("\n" + "="*60)

In [ ]:
# Cell 6: Cleanup - close database connection
conn.close()
print("Database connection closed.")